<a href="https://colab.research.google.com/github/Ch256981/IMLP347/blob/main/HW1_Ko_I_Hung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BUDT 758J - HW1

##I-Hung Ko

In [2]:
# install PySpark
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
# create a SparkSession from pyspark.sql, which is the entry point to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HW1').getOrCreate()

In [4]:
# verify
spark

#Load Dataset

In [55]:
# 703
spark.read.json('hw1_customer.json').createOrReplaceTempView('Customer')
spark.read.json('hw1_order.json').createOrReplaceTempView('Order')
spark.read.json('hw1_orderline.json').createOrReplaceTempView('OrderLine')
spark.read.json('hw1_product.json').createOrReplaceTempView('Product')

In [56]:
# 704
import pandas as pd
pd_customer = pd.read_json('hw1_customer.json')
pd_order = pd.read_json('hw1_order.json')
pd_orderline = pd.read_json('hw1_orderline.json')
pd_product = pd.read_json('hw1_product.json')

In [57]:
# 758J
sp_customer = spark.read.json('hw1_customer.json')
sp_order = spark.read.json('hw1_order.json')
sp_orderline = spark.read.json('hw1_orderline.json')
sp_product = spark.read.json('hw1_product.json')

#Data Manipulation

#1. How many customers in each of the state, New Jersey or New York?

In [58]:
# 703
spark.sql('''SELECT c.customerState, COUNT(c.customerId) AS `Count of Customers`
       FROM Customer c
       WHERE customerState IN ('NJ', 'NY')
       GROUP BY c.customerState''').show()

+-------------+------------------+
|customerState|Count of Customers|
+-------------+------------------+
|           NJ|                 2|
|           NY|                 1|
+-------------+------------------+



In [59]:
# 704
pd_customer.groupby('customerState')['customerId'].count().reset_index()\
      .query('customerState in [\'NJ\', \'NY\']')\
      .rename(columns={'customerId': 'Count of Customers'})

,customerState,Count of Customers
5,NJ,2
6,NY,1


In [80]:
# 758J
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.col.html
from pyspark.sql.functions import col
sp_customer.filter('customerState IN (\'NJ\', \'NY\')')\
      .groupBy('customerState')\
      .count()\
      .select('customerState', col('count').alias('Count of Customers')).show()

+-------------+------------------+
|customerState|Count of Customers|
+-------------+------------------+
|           NJ|                 2|
|           NY|                 1|
+-------------+------------------+



#2. How many orders in each month?

In [62]:
# 703
spark.sql('''SELECT Month(o.orderDate) AS `Month`, COUNT (o.orderId) AS `Count of Orders`
       FROM Order o
       GROUP BY Month(orderDate)
       ORDER BY Month''').show()

+-----+---------------+
|Month|Count of Orders|
+-----+---------------+
|   10|              8|
|   11|              2|
+-----+---------------+



In [63]:
# 704
pd_order.groupby(pd.to_datetime(pd_order['orderDate']).dt.month)['orderId']\
    .count()\
    .reset_index(name='Count of Orders')\
    .rename(columns={'orderDate': 'Month'})

,Month,Count of Orders
0,10,8
1,11,2


In [64]:
# 758J
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.months.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.count.html
from pyspark.sql.functions import month, count
sp_order.select(month('orderDate')\
        .alias('Month'))\
    .groupBy('Month')\
    .agg(count('*')\
       .alias('Count of Orders')).show()

+-----+---------------+
|Month|Count of Orders|
+-----+---------------+
|   10|              8|
|   11|              2|
+-----+---------------+



#3. What is the product id with total ordered quantity greater than 4 in the ascending order of product id?

In [75]:
# 703
spark.sql('''SELECT *
       FROM (
         SELECT l.productId, SUM(l.orderedQuantity) AS `Total Ordered Quantity`
         FROM Orderline l
         GROUP BY l.productId ) t
       WHERE `Total Ordered Quantity` > 4
       ORDER BY t.productId ASC''').show()

+---------+----------------------+
|productId|Total Ordered Quantity|
+---------+----------------------+
|        1|                     5|
|        3|                    11|
|        4|                     7|
|        7|                     5|
|        8|                    15|
+---------+----------------------+



In [71]:
# 704
pd_orderline.groupby('productId')['orderedQuantity'].sum().reset_index()\
      .query('orderedQuantity > 4').sort_values('productId')\
      .rename(columns={'orderedQuantity': 'Total Ordered Quantity'})

,productId,Total Ordered Quantity
0,1,5
2,3,11
3,4,7
6,7,5
7,8,15


In [72]:
# 758J
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.sum.html
from pyspark.sql.functions import sum
sp_orderline.groupBy('productId')\
      .agg(sum('orderedQuantity')\
        .alias('Total Ordered Quantity'))\
      .filter('sum(orderedQuantity) > 4')\
      .orderBy('productId').show()

+---------+----------------------+
|productId|Total Ordered Quantity|
+---------+----------------------+
|        1|                     5|
|        3|                    11|
|        4|                     7|
|        7|                     5|
|        8|                    15|
+---------+----------------------+



#4. What are the description, finish, and price for all products, if its price is lower than the average price for all products, in the descending order of price difference?

In [78]:
# 703
spark.sql('''SELECT p.productDescription, p.productFinish, p.productPrice,
       ROUND((SELECT AVG(productPrice) FROM Product p) - p.productPrice, 2) AS `PriceDifference`
       FROM Product p
       WHERE p.productPrice < (SELECT AVG(productPrice) FROM Product p)
       ORDER BY `PriceDifference` DESC''').show()

+--------------------+-------------+------------+---------------+
|  productDescription|productFinish|productPrice|PriceDifference|
+--------------------+-------------+------------+---------------+
|         48 Bookcase|       Walnut|         150|         384.63|
|          Nightstand|       Cherry|         150|         384.63|
|    Cherry End Table|       Cherry|         175|         359.63|
|         48 Bookcase|          Oak|         175|         359.63|
| Birch Coffee Tables|        Birch|         200|         334.63|
|         96 Bookcase|          Oak|         200|         334.63|
|         96 Bookcase|       Walnut|         225|         309.63|
|      Pine End Table|         Pine|         256|         278.63|
|       Writer's Desk|        Birch|         300|         234.63|
|       Writer's Desk|          Oak|         325|         209.63|
|High Back Leather...|      Leather|         362|         172.63|
|    4-Drawer Dresser|          Oak|         500|          34.63|
+---------

In [79]:
# 704
pd_product[pd_product['productPrice'] < pd_product['productPrice'].mean().round(2)]\
  .sort_values(by='productPrice', ascending=True)\
  .assign(PriceDifference=pd_product['productPrice'].mean().round(2) - pd_product['productPrice'])\
  [['productDescription', 'productFinish', 'productPrice', 'PriceDifference']]

,productDescription,productFinish,productPrice,PriceDifference
6,48 Bookcase,Walnut,150,384.63
12,Nightstand,Cherry,150,384.63
0,Cherry End Table,Cherry,175,359.63
7,48 Bookcase,Oak,175,359.63
1,Birch Coffee Tables,Birch,200,334.63
9,96 Bookcase,Oak,200,334.63
8,96 Bookcase,Walnut,225,309.63
18,Pine End Table,Pine,256,278.63
13,Writer's Desk,Birch,300,234.63
4,Writer's Desk,Oak,325,209.63


In [86]:
# 758J
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.avg.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.round.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.col.html
from pyspark.sql.functions import avg, round, col
average_price = sp_product.select(avg('productPrice')).first()[0]
sp_product.filter(sp_product['productPrice'] < average_price)\
     .withColumn('PriceDifference', round(average_price - col('productPrice'), 2))\
     .orderBy(col('PriceDifference').desc())\
     .select('productDescription', 'productFinish', 'productPrice', 'PriceDifference').show()

+--------------------+-------------+------------+---------------+
|  productDescription|productFinish|productPrice|PriceDifference|
+--------------------+-------------+------------+---------------+
|         48 Bookcase|       Walnut|         150|         384.63|
|          Nightstand|       Cherry|         150|         384.63|
|    Cherry End Table|       Cherry|         175|         359.63|
|         48 Bookcase|          Oak|         175|         359.63|
| Birch Coffee Tables|        Birch|         200|         334.63|
|         96 Bookcase|          Oak|         200|         334.63|
|         96 Bookcase|       Walnut|         225|         309.63|
|      Pine End Table|         Pine|         256|         278.63|
|       Writer's Desk|        Birch|         300|         234.63|
|       Writer's Desk|          Oak|         325|         209.63|
|High Back Leather...|      Leather|         362|         172.63|
|    4-Drawer Dresser|          Oak|         500|          34.63|
+---------